# JuSpyce API test: null maps generation & permutation test

In [1]:
import sys
import os
from glob import glob
import pathlib
import numpy as np
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce
from juspyce.stats import *
from juspyce.utils import *

/Users/llotter/projects/juspyce


## Load JuSpyce data from test_juspyce.fit.ipynb

In [2]:
juspyce_vol = JuSpyce.from_pickle(os.path.join(wd, "testing", "test_juspyce_vol.pkl.gz"))

INFO:juspyce.api:Loaded complete object from /Users/llotter/projects/juspyce/testing/test_juspyce_vol.pkl.gz.


## Permutation based on null maps

Uses the method from brainsmash based on variograms to correct for spatial autocorrelation. Distance maps are based on geodesic distances for surface parcellations and euclidean distances for volumetric parcellations.

One has to choose a method for which empirical p-values will be calculated. If the `JuSpyce.permute_maps()` method is then applied again for computation of another metric, the already existing null maps will be used (this behavior can be turned off).

In [3]:
# everything below that is set to None will be set to customs or taken from the .fit() method
_ = juspyce_vol.permute_maps(
    method="dominance", # which method
    comparison=None, # use dataframe from .compare() function
    permute="X", # null maps for which dataset? Can be 'X' or 'Y'
    null_method="variogram", # "variogram" -> brainsmash, "random" -> np.random
    null_maps=None, # directly provide null maps. Must be dict with keys corresponding to the maps
                    # for which null data is generated (e.g., "mGluR5" or so)
    use_null_maps=True, # re-use null maps if already computed for another prediction method
    dist_mat=None, # custom distance matrix -> must be ndarray with shape (n_parcels, n_parcels) 
                   # or tuple of those ndarrays if parcellation is tuple of giftis
    n_perm=1000, # number of permutations (= number of null maps)
    parcellation=None, parc_space=None, parc_hemi=None, # parcellation data, usually set at .fit()
    centroids=False, # compute distance matrices between parcel centroids? -> faster
    r_to_z=True, adjust_r2=True, mlr_individual=True, # settings for prediction (see other notebook)
    p_tail=None, # which-sided p-value? Must be dict with e.g. 
                 # {"spearman":"two"} or 
                 # {"dominance_total":"upper", "dominance_relative":"upper", 
                 #  "dominance_full_r2":"upper", "dominance_individual":"upper"}, etc.
    n_proc=None, # number of processes
    n_proc_predict=1, # number of processes for prediction method -> multiplies itself with n_proc!
    seed=None, # seed for reproducability
    store=True)

INFO:juspyce.api:Running 'true' prediction (method = 'dominance').


Predicting (dominance, 1 proc):   0%|          | 0/28 [00:00<?, ?it/s]

INFO:juspyce.api:No null maps found.
INFO:juspyce.api:Generating null maps for 'X' data (n = 1000, null_method = 'variogram').
INFO:juspyce.nulls:Null map generation: Assuming n = 7 data vector(s) for n = 116 parcels.
INFO:juspyce.nulls:Loaded parcellation (parc_space = 'MNI152').
INFO:juspyce.nulls:Calculating distance matrix/matrices (euclidean).


Generating null maps (8 proc):   0%|          | 0/7 [00:00<?, ?it/s]

INFO:juspyce.nulls:Null data generation finished.


Null predictions (dominance, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = '{'dominance_total': 'upper', 'dominance_individual': 'upper', 'dominance_relative': 'upper', 'dominance_full_r2': 'upper'}').


### Print result

All results are stored in a dict as `JuSpyce.p_predictions["prediction_name"]` which corresponds to `JuSpyce.predictions["prediction_name"]`

In [4]:
for dom in ["dominance_full_r2", "dominance_relative", "dominance_total", "dominance_individual"]:
    print(dom)
    display(juspyce_vol.p_predictions[dom])

dominance_full_r2


,dominance_full_r2
control,0.007
touch,0.126
interoception,0.056
learning,0.001
attention,0.767
language,0.503
interaction,0.112
inhibition,0.064
somatosensory,0.122
decision,0.001


dominance_relative


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.203,0.696,0.410,0.387,0.263,0.491,0.389
touch,0.264,0.568,0.162,0.364,0.514,0.577,0.323
interoception,0.685,0.244,0.130,0.264,0.672,0.549,0.419
learning,0.465,0.519,0.217,0.425,0.482,0.499,0.168
attention,0.858,0.476,0.642,0.004,0.702,0.901,0.497
language,0.050,0.815,0.276,0.678,0.455,0.389,0.810
interaction,0.541,0.613,0.376,0.025,0.671,0.698,0.654
inhibition,0.178,0.638,0.386,0.409,0.311,0.396,0.425
somatosensory,0.312,0.545,0.133,0.384,0.523,0.610,0.325
decision,0.572,0.610,0.482,0.299,0.456,0.175,0.186


dominance_total


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.040,0.612,0.259,0.241,0.065,0.375,0.169
touch,0.112,0.460,0.052,0.270,0.387,0.563,0.174
interoception,0.679,0.073,0.030,0.139,0.641,0.584,0.291
learning,0.030,0.090,0.001,0.146,0.066,0.424,0.001
attention,0.856,0.501,0.658,0.004,0.724,0.864,0.523
language,0.064,0.812,0.268,0.653,0.440,0.379,0.820
interaction,0.425,0.524,0.253,0.001,0.628,0.825,0.651
inhibition,0.055,0.525,0.253,0.323,0.144,0.270,0.288
somatosensory,0.140,0.447,0.039,0.287,0.414,0.622,0.180
decision,0.223,0.325,0.184,0.007,0.047,0.001,0.001


dominance_individual


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.039,0.748,0.181,0.579,0.053,0.862,0.934
touch,0.355,0.495,0.438,0.489,0.708,0.461,0.435
interoception,0.701,0.029,0.048,0.036,0.936,0.604,0.134
learning,0.001,0.002,0.001,0.025,0.001,0.768,0.001
attention,0.688,0.381,0.793,0.011,0.750,0.808,0.375
language,0.126,0.750,0.231,0.805,0.643,0.347,0.821
interaction,0.528,0.981,0.160,0.003,0.928,0.704,0.712
inhibition,0.086,0.632,0.643,0.700,0.163,0.585,0.989
somatosensory,0.446,0.446,0.295,0.484,0.820,0.535,0.411
decision,0.023,0.222,0.325,0.001,0.001,0.001,0.001


## Correct p-values

p values can be corrected across dataframes or rows/columns of dataframes using `JuSpyce.correct_p()`. The method will, if not provided differently, loop over all p-value dataframes and apply multiple comparison correction methods from `statsmodels.stats.multitest.multipletests`.

Results will be stored in the `JuSpyce.p_predictions` dict as `JuSpyce.p_predictions["prediction_name--correction_method"]`, e.g., if method is `spearman` and correction is `fdr_bh:` `juspyce_vol.p_predictions["spearman--fdr_bh"]`

In [5]:
juspyce_vol.correct_p(
    analysis="predictions", # one of "predictions" or "comparisons" -> here: predictions
    method="all", # if all, iterate over all dataframes (but calculate values for each individual dataframe)
    mc_alpha=0.05, # alpha treshold, should have no effect
    mc_method="fdr_bh", # correction method passed to statsmodels
    mc_dimension="array") # 'array', 'row' or 'column'
display(juspyce_vol.p_predictions["dominance_total--fdr_bh"])

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.229600,0.732667,0.525064,0.525064,0.287467,0.616452,0.435842
touch,0.354065,0.658778,0.261333,0.525064,0.616452,0.707359,0.442909
interoception,0.747663,0.298083,0.210000,0.402889,0.743408,0.715400,0.525064
learning,0.210000,0.332830,0.015077,0.402889,0.287467,0.640769,0.015077
attention,0.871337,0.677214,0.743448,0.041263,0.779739,0.872907,0.681457
language,0.287467,0.846553,0.525064,0.743448,0.649895,0.616452,0.850370
interaction,0.640769,0.681457,0.525064,0.015077,0.732667,0.851053,0.743448
inhibition,0.269500,0.681457,0.525064,0.567368,0.402889,0.525064,0.525064
somatosensory,0.402889,0.653821,0.229600,0.525064,0.640769,0.732667,0.446582
decision,0.502391,0.567368,0.450800,0.068600,0.255889,0.015077,0.015077


## Use "comparison" Y data

In [6]:
# groups
n_Y = juspyce_vol.Y.shape[0]
groups = [0] * int(n_Y/2) + [1] * int(n_Y/2)
print(n_Y, groups)

28 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
# apply comparison
juspyce_vol.compare(
    comparison="diff(mean(A),mean(B))",
    groups=groups
)
# run prediction & permutation
_ = juspyce_vol.permute_maps(
    method="spearman",
    comparison="diff(mean(A),mean(B))", 
    permute="X",
    n_perm=1000,
    store=True)

INFO:juspyce.api:Subtracting parcelwise mean of B from mean of A: new Y = mean(Y[A]) - mean(Y[B]).
INFO:juspyce.api:Running 'true' prediction (method = 'spearman').


Predicting (spearman, 1 proc):   0%|          | 0/1 [00:00<?, ?it/s]

Null predictions (spearman, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = '{'spearman': 'two'}').


In [8]:
juspyce_vol.p_predictions["diff(mean(A),mean(B))-spearman"]

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.282,0.656,0.712,0.214,0.29,0.67,0.776


In [9]:
juspyce_vol.correct_p(analysis="predictions")
display(juspyce_vol.p_predictions["diff(mean(A),mean(B))-spearman--fdr_bh"])

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.676667,0.776,0.776,0.676667,0.676667,0.776,0.776
